In [ ]:
# Evaluate Models

# Import necessary libraries
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
from src.models.gan_model import Generator, Discriminator  # Assuming GAN models are defined in src/models/gan_model.py
from src.models.dqn_model import DQN  # Assuming DQN model is defined in src/models/dqn_model.py

# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the processed test data
test_data_path = '../data/processed/test_user_profiles.csv'
test_df = pd.read_csv(test_data_path)

# Ensure the DataFrame is correctly loaded
print(f"Test Data Shape: {test_df.shape}")
print(test_df.head())

# Convert the DataFrame to a numpy array and then to a torch tensor
test_data = torch.tensor(test_df.values, dtype=torch.float32)

# Define DataLoader for test data
batch_size = 64
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Load trained GAN models
generator = Generator(input_dim=100, output_dim=test_data.shape[1]).to(device)
discriminator = Discriminator(input_dim=test_data.shape[1]).to(device)

generator.load_state_dict(torch.load('../models/generator.pth'))
discriminator.load_state_dict(torch.load('../models/discriminator.pth'))

generator.eval()
discriminator.eval()

# Evaluate GAN-generated data
generated_data = []
with torch.no_grad():
    for _ in range(len(test_loader)):
        noise = torch.randn(batch_size, 100).to(device)
        fake_data = generator(noise)
        generated_data.append(fake_data.cpu().detach().numpy())

generated_data = np.concatenate(generated_data, axis=0)
generated_df = pd.DataFrame(generated_data, columns=test_df.columns)

# Ensure the generated DataFrame is correctly created
print(f"Generated Data Shape: {generated_df.shape}")
print(generated_df.head())

# Evaluate GAN performance
gan_mse = mean_squared_error(test_df.values, generated_df.values)
gan_mae = mean_absolute_error(test_df.values, generated_df.values)
gan_r2 = r2_score(test_df.values, generated_df.values)

print(f"GAN Mean Squared Error (MSE): {gan_mse}")
print(f"GAN Mean Absolute Error (MAE): {gan_mae}")
print(f"GAN R^2 Score: {gan_r2}")

# Load trained RL model (DQN)
dqn = DQN(input_dim=test_data.shape[1] - 1, output_dim=1).to(device)
dqn.load_state_dict(torch.load('../models/dqn.pth'))
dqn.eval()

# Evaluate RL performance
scaler = MinMaxScaler()
test_data_scaled = scaler.fit_transform(test_df.iloc[:, :-1].values)

state = torch.FloatTensor(test_data_scaled).to(device)
with torch.no_grad():
    predicted_ratings = dqn(state).cpu().detach().numpy()

# Ensure the predicted ratings are correctly computed
print(f"Predicted Ratings Shape: {predicted_ratings.shape}")
print(predicted_ratings[:5])

# Evaluate RL performance
rl_mse = mean_squared_error(test_df.iloc[:, -1].values, predicted_ratings)
rl_mae = mean_absolute_error(test_df.iloc[:, -1].values, predicted_ratings)
rl_r2 = r2_score(test_df.iloc[:, -1].values, predicted_ratings)

print(f"RL Mean Squared Error (MSE): {rl_mse}")
print(f"RL Mean Absolute Error (MAE): {rl_mae}")
print(f"RL R^2 Score: {rl_r2}")

# Additional evaluation metrics or visualizations can be added here

# Visualize actual vs predicted ratings for RL model
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.scatter(test_df.iloc[:, -1].values, predicted_ratings, alpha=0.3)
plt.xlabel("Actual Ratings")
plt.ylabel("Predicted Ratings")
plt.title("Actual vs Predicted Ratings (RL Model)")
plt.show()

# Visualize distribution of actual and GAN-generated data
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.hist(test_df.values.flatten(), bins=50, alpha=0.7, label='Real Data')
plt.title("Distribution of Real Data")
plt.legend()

plt.subplot(1, 2, 2)
plt.hist(generated_df.values.flatten(), bins=50, alpha=0.7, color='orange', label='GAN Generated Data')
plt.title("Distribution of GAN Generated Data")
plt.legend()

plt.show()

# Save the evaluation metrics to a CSV file
evaluation_metrics = {
    'Model': ['GAN', 'RL'],
    'MSE': [gan_mse, rl_mse],
    'MAE': [gan_mae, rl_mae],
    'R2': [gan_r2, rl_r2]
}
metrics_df = pd.DataFrame(evaluation_metrics)
metrics_df.to_csv('../data/processed/evaluation_metrics.csv', index=False)
print("Evaluation metrics saved to '../data/processed/evaluation_metrics.csv'")
